In [1]:
from pythainlp import word_tokenize, Tokenizer
from pythainlp.util.trie import Trie
from pythainlp.util import normalize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus.common import thai_words
import numpy as np
import pandas as pd
import re

In [2]:
stopwords = list(thai_stopwords())

In [3]:
import pickle

In [4]:
df = pickle.load(open("dataframe.pkl", "rb"))
df

,text,sentiment,pos,neg,text_tokens
0,เกลียด ชอบหลอกให้สั่งซื้อ ที่ไหนได้ไม่มีสิ้นค้า,neg,0,1,เกลียด ชอบ หลอก สั่งซื้อ ที่ไหนได้ ค้า
1,สินค้าด้อยคุณภาพ,neg,0,1,สินค้า ด้อย คุณภาพ
2,สินค้าหมดทำไมไม่แจ้งขึ้นว่าหมดอะ.กดใส่ตะกร้าไป...,neg,0,1,สินค้า แจ้ง อะ .กด ใส่ ตะกร้า . เซ็ง เรย
3,อย่าทำแบบนี้กับใครอีกนะค่ะแย่มากค่ะ,neg,0,1,อย่า ทำ แบบนี้ แย่มาก
4,คือเเบบผิดหวังมาก เเย่ที่สุดได้ของมาฝาเเตก เเต...,neg,0,1,ผิดหวัง แย่ ฝา แตก แตก แตก จบ แยก 😢
...,...,...,...,...,...
123,สินค้ามีตำหนิ แย่มากๆ ไม่ไหวๆเลย ถ้ามีตำหนิก็บ...,neg,0,1,สินค้า มีตำหนิ แย่มาก ไหว มีตำหนิ ทำใจ ซื้อ
124,เปราะบางมากความแข็งแรงน้อย มันง่ายต่อการแตกหัก,neg,0,1,เปราะบาง ความแข็งแรง แตกหัก
125,ขนส่งก็ช้าสินค้ามีตำหนิแพ็คสินค้าไม่ดีมีรอยบุบ,neg,0,1,ขนส่ง สินค้า มีตำหนิ แพ็ค สินค้า ดี รอย บุบ
126,สินค้าเหมือนของปลอม เนื่อครีมเหมือนซอง 10 บาท ...,neg,0,1,สินค้า เหมือน ของปลอม เนื่อ ครีม เหมือน ซอง 10...


In [5]:
def text_preprocess(text):
    text = normalize(text)
    text = re.sub('\s+', ' ', text)
    regex = re.compile(r"[^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''")
    text = regex.sub('', text)
    new_words = {"ไม่มี", "ไม่ดี"}
    custom_words_list = set(thai_words())
    custom_words_list.update(new_words)
    trie = Trie(custom_words_list)

    custom_tokenizer = Tokenizer(custom_dict=trie, engine='newmm')
    output = custom_tokenizer.word_tokenize(text)

    symbols = {" ", "  "}
    last_value = [i for i in output if  i not in symbols and len(i) > 2]

    last_list = " ".join(i for i in last_value if i not in stopwords)
    return last_list

In [6]:
# run this code if you havent tokenized yet or wanna use other dataframe. dont forget to match ur dataframe cols and code correctly
# df['text_tokens'] = df['text'].apply(text_preprocess)

In [7]:
from sklearn.model_selection import train_test_split

x = df['text_tokens']
y = df['sentiment']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 101)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(analyzer=lambda x:x.split(' ')) #analyse by space that we tokenized

In [9]:
x = cvec.fit_transform(x_train)

In [10]:
cvec.transform(x_train)

<89x264 sparse matrix of type '<class 'numpy.int64'>'
	with 501 stored elements in Compressed Sparse Row format>

In [11]:
# cvec.vocabulary_

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
model = LogisticRegression()
model.fit(x, y_train)

LogisticRegression()

In [14]:
test_bow = cvec.transform(x_test)
model.score(test_bow, y_test)

0.7948717948717948

In [15]:
pred = model.predict(test_bow)

In [16]:
from sklearn.metrics import classification_report

print(classification_report(pred, y_test))

              precision    recall  f1-score   support

         neg       0.91      0.78      0.84        27
         pos       0.62      0.83      0.71        12

    accuracy                           0.79        39
   macro avg       0.77      0.81      0.78        39
weighted avg       0.82      0.79      0.80        39



In [17]:
def pred_ana(my_text):
    my_tokens = text_preprocess(my_text)
    my_bow = cvec.transform(pd.Series([my_tokens]))
    my_predictions = model.predict(my_bow)
    my_proba = model.predict_proba(my_bow)
    print(my_proba)
    print(f'Sentiment : {" ".join(x for x in my_predictions)}')

In [18]:
pred_ana('วันนี้อากาศแย่มาก ฟ้าฝนไม่เป็นใจ เมื่อไหร่จะถึงวันเสาร์สักที')

[[0.85773402 0.14226598]]
Sentiment : neg
